In [ ]:
from elasticsearch import Elasticsearch, helpers
import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import date2num
import matplotlib.ticker as ticker

es = Elasticsearch("http://es:9200")
product = 'LST'
product_index = product.lower()

def draw(searched):
    def get_value(hit):
        return hit['inner_hits']['distance']['hits']['hits'][0]['_source']['value']
    
    def get_date(hit):
        return hit['inner_hits']['distance']['hits']['hits'][0]['_source']['observation_datetime']
    
    x = [datetime.datetime.fromisoformat(d) for d in map(get_date, searched['hits']['hits'])]
    y = np.array(list(map(get_value, searched['hits']['hits']))) - 273.15

    figure = plt.figure(1, figsize=(8,4))
    axes = figure.add_subplot(111, title='LST', ylabel='[deg C]')
    axes.plot(x, y, 'o-')
    axes.xaxis.set_major_locator(ticker.FixedLocator(date2num(x)))
    axes.set_xticklabels([d.strftime('%Y/%m/%d %H:%M:%S') for d in x], rotation=320, ha='left')

In [ ]:
point = [75.05859375000006, 62.0043408521]

searched = es.search(index=product_index, body={
    "query": {
        "bool": {
            "must": {
                "match_all": {}
            },
            "filter": {
                "geo_distance": {
                    "distance": "1km",
                    "location": point
                }
            }
        }
    },
    "sort": [
        {
            "observation_datetime": {
                "order": "asc"
            }
        }
    ],
    "collapse": {
        "field": "granule_id",
        "inner_hits": {
            "name": "distance",
            "size": 1,
            "sort": [
                {
                    "_geo_distance": {
                        "location": point
                    }
                }
            ]
        }
    }
})
print(searched)

In [ ]:
draw(searched)

In [ ]:
point = [75.05859375000006, 62.0043408521]

searched = es.search(index=product_index, body={
    "query": {
        "bool": {
            "must": [
                {"term":{"qa_flags": 11}}
            ],
            "must_not": [
                {"term":{"qa_flags": 6}}
            ],
            "filter": {
                "geo_distance": {
                    "distance": "1km",
                    "location": point
                }
            }
        }
    },
    "sort": [
        {
            "observation_datetime": {
                "order": "asc"
            }
        }
    ],
    "collapse": {
        "field": "granule_id",
        "inner_hits": {
            "name": "distance",
            "size": 1,
            "sort": [
                {
                    "_geo_distance": {
                        "location": point
                    }
                }
            ]
        }
    }
})
print(searched)

In [ ]:
draw(searched)